In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import pulp

class TravelPartnerOptimizer:
    def __init__(self, root):
        self.root = root
        self.root.title("Travel Partner Optimizer")  #視窗標題

        #權重設定
        self.weights = {
            'route_planning': 1.5,
            'punctuality': 1.0,
            'adventurousness': 2.0, 
            'complaining': -2.0,
            'relation_score': 4.0  #令關係約束較容易影響選擇結果
        }

        self.create_initial_widgets()

    def create_initial_widgets(self):
        #初始輸入標題
        tk.Label(self.root, text="Total number of friends:").grid(row=0, column=0)
        tk.Label(self.root, text="Number of friends to select:").grid(row=1, column=0)
        
        #總人數
        self.total_friends_entry = tk.Entry(self.root)
        self.total_friends_entry.grid(row=0, column=1)
        #欲挑選人數
        self.num_to_select_entry = tk.Entry(self.root)
        self.num_to_select_entry.grid(row=1, column=1)

        #確定按鈕
        tk.Button(self.root, text="Submit", command=self.submit_initial_info).grid(row=2, column=0, columnspan=2)

    def submit_initial_info(self):
        try:
            #獲取輸入的總朋友數和欲選擇的人數
            self.total_friends = int(self.total_friends_entry.get())
            self.num_to_select = int(self.num_to_select_entry.get())
            
            #確認欲挑選人數小於總人數
            if self.num_to_select > self.total_friends:
                messagebox.showerror("Error", "Number of friends to select cannot be greater than the total number of friends.")
                return

            #清除初始輸入畫面
            for widget in self.root.winfo_children():
                widget.destroy()

            self.create_main_widgets()
        except ValueError:
            messagebox.showerror("Error", "Please enter valid integers for the number of friends.")

    def create_main_widgets(self):
        #建立畫布和滾動條
        self.canvas = tk.Canvas(self.root)
        self.scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(
                scrollregion=self.canvas.bbox("all")
            )
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.grid(row=0, column=0, sticky="nsew")
        self.scrollbar.grid(row=0, column=1, sticky="ns")

        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        #標題
        tk.Label(self.scrollable_frame, text="Friend").grid(row=0, column=0)
        tk.Label(self.scrollable_frame, text="Route Planning").grid(row=0, column=1)
        tk.Label(self.scrollable_frame, text="Punctuality").grid(row=0, column=2)
        tk.Label(self.scrollable_frame, text="Adventurousness").grid(row=0, column=3)
        tk.Label(self.scrollable_frame, text="Complaining").grid(row=0, column=4)

        #初始化輸入字段
        self.friend_entries = []
        self.route_planning_entries = []
        self.punctuality_entries = []
        self.adventurousness_entries = []
        self.complaining_entries = []

        for i in range(self.total_friends):
            #名字
            friend_entry = tk.Entry(self.scrollable_frame)
            friend_entry.grid(row=i + 1, column=0)
            self.friend_entries.append(friend_entry)
            
            #輸入四項特質的評分
            route_planning_entry = tk.Entry(self.scrollable_frame)
            route_planning_entry.grid(row=i + 1, column=1)
            self.route_planning_entries.append(route_planning_entry)

            punctuality_entry = tk.Entry(self.scrollable_frame)
            punctuality_entry.grid(row=i + 1, column=2)
            self.punctuality_entries.append(punctuality_entry)

            adventurousness_entry = tk.Entry(self.scrollable_frame)
            adventurousness_entry.grid(row=i + 1, column=3)
            self.adventurousness_entries.append(adventurousness_entry)

            complaining_entry = tk.Entry(self.scrollable_frame)
            complaining_entry.grid(row=i + 1, column=4)
            self.complaining_entries.append(complaining_entry)

        #下一步按鈕
        tk.Button(self.scrollable_frame, text="Next", command=self.submit_friends_info).grid(row=self.total_friends + 1, column=0, columnspan=5)

    def submit_friends_info(self):
        self.friends = {}
        friend_names = set()
        try:
            for i in range(self.total_friends):
                name = self.friend_entries[i].get()
                if name:
                    #檢查輸入的名字是否重複
                    if name in friend_names:
                        messagebox.showerror("Error", "Duplicate friend names are not allowed.")
                        return
                    friend_names.add(name)
                    self.friends[name] = {
                        'route_planning': int(self.route_planning_entries[i].get()),
                        'punctuality': int(self.punctuality_entries[i].get()),
                        'adventurousness': int(self.adventurousness_entries[i].get()),
                        'complaining': int(self.complaining_entries[i].get())
                    }

                    #檢查評分是否在0-10之間
                    for trait in self.friends[name].values():
                        if not 0 <= trait <= 10:
                            messagebox.showerror("Error", "Scores must be integers between 0 and 10.")
                            return

            if len(self.friends) < self.num_to_select:
                messagebox.showerror("Error", "Number of friends to select cannot be greater than the number of entered friends.")
                return

            #清除當前畫面
            for widget in self.root.winfo_children():
                widget.destroy()

            self.create_relation_widgets()
        except ValueError:
            messagebox.showerror("Error", "Scores must be integers between 0 and 10.")

    def create_relation_widgets(self):
        #建立畫布和滾動條
        self.canvas = tk.Canvas(self.root)
        self.scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ttk.Frame(self.canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: self.canvas.configure(
                scrollregion=self.canvas.bbox("all")
            )
        )

        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        self.canvas.grid(row=0, column=0, sticky="nsew")
        self.scrollbar.grid(row=0, column=1, sticky="ns")

        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        #關係約束標題
        tk.Label(self.scrollable_frame, text="Relations (Friend1, Friend2, Score)").grid(row=0, column=0, columnspan=11)
        self.relation_entries = []

        #定義兩兩之間的關係約束
        row_offset = 1
        friend_names = list(self.friends.keys())
        for i in range(len(friend_names)):
            for j in range(i + 1, len(friend_names)):
                relation_frame = tk.Frame(self.scrollable_frame)
                relation_frame.grid(row=row_offset, column=0, columnspan=11)

                tk.Label(relation_frame, text=f"{friend_names[i]} - {friend_names[j]}:").grid(row=0, column=0)
                relation_score = tk.IntVar(value=0)
                for score in range(-5, 6):  #評分範圍-5~5
                    tk.Radiobutton(relation_frame, text=str(score), variable=relation_score, value=score).grid(row=0, column=1 + score + 5)
                self.relation_entries.append(((friend_names[i], friend_names[j]), relation_score))
                row_offset += 1

        # 優化按鈕
        tk.Button(self.scrollable_frame, text="Optimize", command=self.optimize).grid(row=row_offset, column=0, columnspan=11)

    def optimize(self):
        try:
            relation_scores = {}
            
            #從relation_entries中提取評分分數
            for (friend1, friend2), score_var in self.relation_entries:
                score = score_var.get()
                relation_scores[(friend1, friend2)] = score
            
            #定義一個最大化問題
            prob = pulp.LpProblem("TravelPartnerSelection", pulp.LpMaximize)
            friend_vars = pulp.LpVariable.dicts("Friend", self.friends.keys(), cat='Binary')
            pair_vars = pulp.LpVariable.dicts("Pair", relation_scores.keys(), cat='Binary')
            
            #定義目標函數
            prob += pulp.lpSum([
                friend_vars[f] * (
                    self.weights['route_planning'] * self.friends[f]['route_planning'] +
                    self.weights['punctuality'] * self.friends[f]['punctuality'] +
                    self.weights['adventurousness'] * self.friends[f]['adventurousness'] +
                    self.weights['complaining'] * self.friends[f]['complaining']
                ) for f in self.friends
            ]) + pulp.lpSum([
                self.weights['relation_score'] * relation_scores[pair] * pair_vars[pair] for pair in relation_scores
            ])

            #選出的人數必須等於用戶指定的數量
            prob += pulp.lpSum([friend_vars[f] for f in self.friends]) == self.num_to_select

            #對於每對朋友添加約束條件
            for pair in relation_scores:
                prob += pair_vars[pair] <= friend_vars[pair[0]]
                prob += pair_vars[pair] <= friend_vars[pair[1]]
                prob += pair_vars[pair] >= friend_vars[pair[0]] + friend_vars[pair[1]] - 1

            #求解問題
            prob.solve()

            selected_friends = [f for f in self.friends if friend_vars[f].value() == 1.0]

            #顯示最佳化結果
            messagebox.showinfo("Optimization Result", f"Selected Friends: {', '.join(selected_friends)}")

        
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = TravelPartnerOptimizer(root)
    root.mainloop()